In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder,StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [42]:
# from google.colab import drive
# drive.mount('/content/drive')

In [43]:
training_data = pd.read_csv(r'C:\Users\muyan\Desktop\DATA602-Final-Project-1\DataForFinal\DataForFinal\data_afterNorm.csv',index_col=0)#/content/drive/MyDrive/602/DataForFinal/data_afterNorm.csv
training_data

,new/used,milage,price,drive mode,fuel type,accident history,one owner?,personal use,brand year,brand,fuel consumption min,fuel consumption max
0,1,-6.649190e-01,38400.000000,RWD,Gasoline,At least 1 accident or damage reported,No,No,-0.671587,other,-3.208156e-01,-0.035195
1,1,3.835387e-16,34637.216663,AWD,Gasoline,None reported,Yes,Yes,0.500749,Ford,6.173207e-16,0.000000
2,1,-5.721175e-01,32994.000000,4WD,Gasoline,At least 1 accident or damage reported,Yes,Yes,0.735217,Nissan,2.670508e-02,-0.035195
3,1,-4.026447e-01,62500.000000,4WD,Gasoline,None reported,No,Yes,0.500749,Jeep,-1.363378e+00,-1.365423
4,1,-9.364972e-01,44995.000000,AWD,Gasoline,None reported,Yes,Yes,0.735217,Cadillac,6.173207e-16,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
154783,0,-1.200300e+00,43370.000000,FWD,Gasoline,None reported,No,No,0.969684,Acura,2.004654e-01,-0.035195
154784,1,-1.326179e+00,86998.000000,4WD,Gasoline,None reported,Yes,Yes,0.969684,Land,-4.945759e-01,-0.700309
154785,1,1.261273e+00,14599.000000,FWD,Gasoline,None reported,No,Yes,-0.906055,Hyundai,-4.945759e-01,-0.367752
154786,1,-4.667268e-02,30775.000000,AWD,Gasoline,None reported,No,Yes,-0.671587,Lexus,2.004654e-01,0.131083


In [44]:
# parameters and encoders used to preprocess the test data
parameters = {}
parameters['new/used'] = training_data['new/used'].mode()[0]
parameters['milage'] = training_data['milage'].mean()
parameters['drive mode'] = training_data['drive mode'].mode()[0]
parameters['fuel type'] = training_data['fuel type'].mode()[0]
parameters['accident history'] = training_data['accident history'].mode()[0]
parameters['one owner?'] = training_data['one owner?'].mode()[0]
parameters['personal use'] = training_data['personal use'].mode()[0]
parameters['brand year'] = training_data['brand year'].mode()[0]
parameters['brand'] = training_data['brand'].mode()[0]
parameters['fuel consumptiion min'] = training_data['fuel consumption min'].mean()
parameters['fuel consumptiion max'] = training_data['fuel consumption min'].mean()
parameters['price'] = training_data['price'].mean()
parameters['top 25 brand'] = training_data['brand'].value_counts()[:25].index

encoders = {}
encoders['drive mode'] = OneHotEncoder().fit(training_data[['drive mode']])
encoders['fuel type'] = OneHotEncoder().fit(training_data[['fuel type']])
encoders['accident history'] = OneHotEncoder().fit(training_data[['accident history']])
encoders['personal use'] = OneHotEncoder().fit(training_data[['personal use']])
encoders['one owner?'] = OneHotEncoder().fit(training_data[['one owner?']])
encoders['brand'] = OneHotEncoder().fit(training_data[['brand']])

c:\Users\muyan\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

c:\Users\muyan\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

c:\Users\muyan\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

c:\Users\muyan\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

c:\Users\muyan\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `

In [45]:
test_data = pd.read_csv(r'C:\Users\muyan\Desktop\DATA602-Final-Project-1\DataForFinal\DataForFinal\full_test_data.csv')#/content/drive/MyDrive/602/DataForFinal/test_data.csv
test_data

,new/used,name,milage,price,id,drive mode,fuel consumption,fuel type,accident history,clean title,one owner?,personal use,factory recall
0,Used,2022 Nissan Pathfinder SL,"28,800 mi.","$32,995",1b6dae71-3160-4c99-8477-edda938c424f,21–27,Four-wheel Drive,Gasoline,At least 1 accident or damage reported,NaN,Yes,Yes,At least 1 open recall reported
1,Used,2021 Jeep Grand Cherokee SRT,"35,230 mi.","$59,995",fa45d263-0171-4222-808f-c2fdf658ff74,13–19,Four-wheel Drive,Gasoline,None reported,NaN,No,Yes,NaN
2,Used,2022 Mercedes-Benz C-Class C 300 4MATIC,"8,997 mi.","$37,998",34be55ba-d511-4c03-bbeb-c73cd066fe29,23–33,All-wheel Drive,Gasoline,None reported,NaN,Yes,Yes,At least 1 open recall reported
3,Cadillac Certified,2020 Cadillac XT5 Luxury,"37,222 mi.","$27,894",8ea65593-75c7-44be-8f55-9b5675f1c6d5,21–28,Front-wheel Drive,Gasoline,None reported,NaN,Yes,Yes,NaN
4,Used,2019 BMW 740 i xDrive,"26,164 mi.","$39,995",fbb66fcb-f26b-4b68-8ba7-25ee766beebf,20–29,All-wheel Drive,Gasoline,None reported,NaN,No,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26653,Acura Certified,2023 Acura RDX A-Spec,"4,966 mi.","$43,370",79a45d1f-1fac-4f7a-a9f4-d9733382926d,22–27,Front-wheel Drive,Gasoline,None reported,NaN,No,No,NaN
26654,Used,2023 Land Rover Defender 75th Limited Edition,190 mi.,"$86,998",2096e10d-8e93-4994-b4b6-00bb1d96f3bb,18–23,Four-wheel Drive,Gasoline,None reported,NaN,Yes,Yes,NaN
26655,Used,2015 Hyundai Santa Fe Limited,"98,361 mi.","$14,599",246d0cac-bd93-4f5a-ac66-514b592f6813,18–25,Front-wheel Drive,Gasoline,None reported,NaN,No,Yes,NaN
26656,Used,2016 Lexus NX 200t Base,"48,736 mi.","$30,775",0466e59e-6ca6-40f0-a51c-1666585db15c,22–28,All-wheel Drive,Gasoline,None reported,Yes,No,Yes,NaN


In [46]:
def dataPreprocess(data, parameters, encoders):
  # new/used
  data['new/used'].fillna(parameters['new/used'],inplace=True)
  data.loc[data['new/used']!='Used','new/used']='Certified'
  data.loc[data['new/used']=='Used','new/used'] = '1'
  data.loc[data['new/used']=='Certified','new/used'] = '0'
  data['new/used'] = pd.to_numeric(data['new/used'],errors='coerce')

  # name
  data['brand year'] = data['name'].str.split(' ').str.get(0)
  data['brand'] = data['name'].str.split(' ').str.get(1)
  data.loc[~data['brand'].isin(parameters['top 25 brand']),'brand'] = 'other'
  data.drop(columns=['name'],axis=1,inplace=True)
  data['brand year'] = pd.to_numeric(data['brand year'])
  data['brand year'].fillna(parameters['brand year'],inplace=True)
  data['brand'].fillna(parameters['brand'],inplace=True)

  # milage
  data['milage'] = data['milage'].str.replace(',','')
  data['milage'] = data['milage'].str.replace(' ','')
  data['milage'] = data['milage'].str.replace('mi.','',regex=False)
  data['milage'] = pd.to_numeric(data['milage'])
  data['milage'].fillna(parameters['milage'],inplace=True)

  # id
  data.drop(columns='id',inplace=True)

  # drive mode
  data.rename(columns={'drive mode':'fuel consumption','fuel consumption':'drive mode'},inplace=True)
  data.loc[data['drive mode']=='Rear-wheel Drive ','drive mode'] = 'RWD '
  data.loc[data['drive mode']=='All-wheel Drive ','drive mode'] = 'AWD '
  data.loc[(data['drive mode']=='Four-wheel Drive ') | (data['drive mode']=='Four Wheel Drive '),'drive mode'] = '4WD '
  data.loc[data['drive mode']=='Front-wheel Drive ','drive mode'] = 'FWD '
  data.loc[data['drive mode']=='– ','drive mode'] = pd.NA
  data.loc[data['drive mode']=='Unknown ','drive mode'] = pd.NA
  data['drive mode'].fillna(parameters['drive mode'],inplace=True)

  # fuel consumption
  data['fuel consumption min'] = data['fuel consumption'].str.split('–').str.get(0)
  data['fuel consumption max'] = data['fuel consumption'].str.split('–').str.get(1)
  data['fuel consumption min'] = pd.to_numeric(data['fuel consumption min'], errors='coerce')
  data['fuel consumption max'] = pd.to_numeric(data['fuel consumption max'], errors='coerce')
  data.drop(columns=['fuel consumption'],axis=1,inplace=True)
  data['fuel consumption min'].fillna(parameters['fuel consumptiion min'],inplace=True)
  data['fuel consumption max'].fillna(parameters['fuel consumptiion max'],inplace=True)

  # fuel type
  data.loc[data['fuel type']=='– ','fuel type'] = pd.NA
  data['fuel type'].fillna(parameters['fuel type'],inplace=True)
  data.loc[~data['fuel type'].isin(['Gasoline ', 'Hybrid ', 'Diesel ', 'E85 Flex Fuel ', '– ']),'fuel type'] = 'other'

  # accident history
  data['accident history'].fillna(data['accident history'].mode()[0],inplace=True)
  data.loc[~data['accident history'].isin(['None reported', 'At least 1 accident or damage reported']),'accident history'] = 'other'

  # clean title
  data.drop(columns='clean title',inplace=True)

  # one owner?
  data['one owner?'].fillna(parameters['one owner?'],inplace=True)
  data.loc[~data['one owner?'].isin(['Yes', 'No']),'one owner?'] = 'other'

  # personal use
  data['personal use'].fillna(parameters['personal use'],inplace=True)
  data.loc[~data['personal use'].isin(['Yes', 'No']),'personal use'] = 'other'

  # factory recall
  data.drop(columns='factory recall',inplace=True)

  data.to_csv(r'C:\Users\muyan\Desktop\DATA602-Final-Project-1\DataForFinal\DataForFinal\data_afterFillna.csv')#/content/drive/MyDrive/602/DataForFinal/data_afterFillna.csv

  # normalization
  data[['milage','brand year','fuel consumption min','fuel consumption max']] = StandardScaler().fit_transform(data[['milage','brand year','fuel consumption min','fuel consumption max']])

  # one hot encode
  drive_mode = encoders['drive mode'].transform(data[['drive mode']]).toarray()
  drive_mode = pd.DataFrame(drive_mode,columns=encoders['drive mode'].get_feature_names_out())

  fuel_type = encoders['fuel type'].transform(data[['fuel type']]).toarray()
  fuel_type = pd.DataFrame(fuel_type,columns=encoders['fuel type'].get_feature_names_out())

  accident_history = encoders['accident history'].transform(data[['accident history']]).toarray()
  accident_history = pd.DataFrame(accident_history,columns=encoders['accident history'].get_feature_names_out())

  one_owner = encoders['one owner?'].transform(data[['one owner?']]).toarray()
  one_owner = pd.DataFrame(one_owner,columns=encoders['one owner?'].get_feature_names_out())

  personal_use = encoders['personal use'].transform(data[['personal use']]).toarray()
  personal_use = pd.DataFrame(personal_use,columns=encoders['personal use'].get_feature_names_out())

  brand = encoders['brand'].transform(data[['brand']]).toarray()
  brand = pd.DataFrame(brand,columns=encoders['brand'].get_feature_names_out())

  data_formodel = pd.concat([data,drive_mode,fuel_type,accident_history,one_owner,personal_use,brand],axis=1)
  data_formodel.drop(columns=['drive mode','fuel type','brand','accident history','one owner?','personal use'],axis=1,inplace=True)

  data_formodel.drop(columns=['price'])
  return data_formodel

In [47]:
data_formodel = dataPreprocess(test_data,parameters,encoders)

c:\Users\muyan\anaconda3\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

c:\Users\muyan\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

c:\Users\muyan\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

c:\Users\muyan\anaconda3\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

c:\Users\muyan\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `

In [48]:
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")
X = data_formodel.loc[:,~data_formodel.columns.isin(['price'])]
X_tensor = torch.tensor(X.values,dtype=torch.float32).to(device)
dataset = TensorDataset(X_tensor)
dataloader = DataLoader(dataset, batch_size=data_formodel.shape[0], shuffle=True)

In [49]:
class Model(nn.Module):
  def __init__(self,input_size: int, output_size: int, drop_p: float):
    super().__init__()
    self.input = nn.Linear(input_size,128)
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(drop_p)
    self.layer_1 = nn.Linear(128,256)
    self.norm1 = nn.BatchNorm1d(256)
    self.layer_2 = nn.Linear(256,512)
    self.norm2 = nn.BatchNorm1d(512)
    self.layer_3 = nn.Linear(512,1024)
    self.norm3 = nn.BatchNorm1d(1024)
    self.layer_4 = nn.Linear(1024,512)
    self.norm4 = nn.BatchNorm1d(512)
    self.layer_5 = nn.Linear(512,256)
    self.norm5 = nn.BatchNorm1d(256)
    self.output = nn.Linear(256,output_size)
  def forward(self,X):
    X = self.input(X)
    X = self.layer_1(X)
    X_res1 = X
    X = self.norm1(X)
    X = self.relu(X)
    X = self.layer_2(X)
    X = self.norm2(X)
    X_res2 = X
    X = self.dropout(X)
    X = self.relu(X)
    X = self.layer_3(X)
    X = self.norm3(X)
    X = self.relu(X)
    X = self.layer_4(X)
    X = self.norm4(X)
    X = X + X_res2
    X = self.dropout(X)
    X = self.relu(X)
    X = self.layer_5(X)
    X = self.norm5(X)
    X = X + X_res1
    X = self.relu(X)
    X = self.output(X)
    return X

In [50]:
model = Model(X.shape[1],1,0)
model.load_state_dict(torch.load(r'C:\Users\muyan\Desktop\DATA602-Final-Project-1\DataForFinal\DataForFinal\model_dict.pt', map_location=torch.device('cpu')))#/content/drive/MyDrive/602/DataForFinal/model_dict.pt

with torch.inference_mode():
  for batch in dataloader:
    X_test = batch
    y_pred_test = model(X_test[0])

In [51]:
test_data['price'] = test_data['price'].str.replace('$','')
test_data['price'] = test_data['price'].str.replace(',','')
test_data['price'] = pd.to_numeric(test_data['price'],errors='coerce')
test_data['price'].fillna(parameters['price'],inplace=True)
y_test = test_data['price']

In [52]:
time = pd.read_csv(r'C:\Users\muyan\Desktop\DATA602-Final-Project-1\DataForFinal\DataForFinal\data_afterFillna.csv',index_col=0) #/content/drive/MyDrive/602/DataForFinal/data_afterFillna.csv
time['price'] = time['price'].str.replace('$','')
time['price'] = time['price'].str.replace(',','')
time['price'] = pd.to_numeric(time['price'],errors='coerce')
time['price'].fillna(time['price'].mean(),inplace=True)
time['brand year'] = time['brand year'].astype(np.int64)
t = time.groupby(by='brand year').mean('price').reset_index()
t = t.loc[t['brand year'] >= 1995]

In [53]:
raw_data = pd.read_csv(r'C:\Users\muyan\Desktop\DATA602-Final-Project-1\DataForFinal\DataForFinal\data_afterFillna.csv')#/content/drive/MyDrive/602/DataForFinal/data_afterFillna.csv

In [54]:
mileage_mean = raw_data['milage'].mean()
mileage_std = raw_data['milage'].std()
year_mean = raw_data['brand year'].mean()
year_std = raw_data['brand year'].std()

In [55]:
raw_features = raw_data[['brand','brand year','milage','accident history','drive mode']]
raw_features['brand year'] = raw_features['brand year'].astype(int)
raw_features.loc[raw_features['drive mode']=='4WD ','drive mode'] = 'Four-wheel Drive'
raw_features.loc[raw_features['drive mode']=='AWD ','drive mode'] = 'All-wheel Drive'
raw_features.loc[raw_features['drive mode']=='FWD ','drive mode'] = 'Front-wheel Drive'
raw_features.loc[raw_features['drive mode']=='RWD ','drive mode'] = 'Rear-wheel Drive'
raw_features

C:\Users\muyan\AppData\Local\Temp\ipykernel_16232\2309633991.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,brand,brand year,milage,accident history,drive mode
0,Nissan,2022,28800.0,At least 1 accident or damage reported,Four-wheel Drive
1,Jeep,2021,35230.0,None reported,Four-wheel Drive
2,Mercedes-Benz,2022,8997.0,None reported,All-wheel Drive
3,Cadillac,2020,37222.0,None reported,Front-wheel Drive
4,BMW,2019,26164.0,None reported,All-wheel Drive
...,...,...,...,...,...
26653,Acura,2023,4966.0,None reported,Front-wheel Drive
26654,Land,2023,190.0,None reported,Four-wheel Drive
26655,Hyundai,2015,98361.0,None reported,Front-wheel Drive
26656,Lexus,2016,48736.0,None reported,All-wheel Drive


In [56]:
brands = training_data['brand'].drop_duplicates()

In [57]:
#!pip install dash

In [61]:
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import plotly.express as px

app = dash.Dash(__name__)

# create dashboard
app.layout = html.Div(style={'backgroundColor': '#f2f3f8', 'padding': '20px'}, children=[

    html.H1("Second Hand Vehicle Dashboard", style={'text-align': 'center', 'color': '#1f3f66'}),

    html.P("Data comes from cars.com", style={'text-align': 'center', 'color': '#333', 'margin-bottom': '30px'}),

    # Data Labels
    html.Div(style={'background-color': '#ffffff', 'padding': '20px', 'border-radius': '10px', 'box-shadow': '0px 0px 10px #ccc', 'margin-top': '20px', 'display': 'flex', 'justify-content': 'space-between'}, children=[
        html.Div(style={'flex': 1, 'text-align': 'center', 'background-color': '#f2f3f8', 'padding': '10px', 'border-radius': '10px', 'margin-right': '10px'}, children=[
            html.H3("Total Number of Vehicles", style={'color': '#a64647'}),
            html.P(f"{len(training_data)} vehicles", style={'color': '#333'})# number of training instances
        ]),

        html.Div(style={'flex': 1, 'text-align': 'center', 'background-color': '#f2f3f8', 'padding': '10px', 'border-radius': '10px', 'margin-right': '10px'}, children=[
            html.H3("Average Mileage of Vehicles", style={'color': '#a64647'}),
            html.P(f"{50506.817} miles", style={'color': '#333'}) # average value of mileage of training instances
        ]),

        html.Div(style={'flex': 1, 'text-align': 'center', 'background-color': '#f2f3f8', 'padding': '10px', 'border-radius': '10px'}, children=[
            html.H3("Percentage of Vehicles without Accidents", style={'color': '#a64647'}),
            html.P(f"{(training_data[training_data['accident history'] == 'None reported'].shape[0] / training_data.shape[0]) * 100:.2f}%") # Percenatge of vehicles without accidents
        ]),
    ]),

    # Plots
    html.Div(style={'display': 'flex', 'justify-content': 'space-between', 'margin-top': '30px'}, children=[
        # First plot
        html.Div(style={'width': '48%', 'background-color': '#ffffff', 'padding': '20px', 'border-radius': '10px', 'box-shadow': '0px 0px 10px #ccc', 'text-align': 'center'}, children=[
            html.H3("Top 10 Brands", style={'color': '#a64647'}),
            dcc.Graph(
                id='top-brands-plot',
                figure=px.bar(
                    training_data['brand'].value_counts().nlargest(10),#Top 10 brands
                    color_discrete_sequence=['#8cb5d3'],
                    labels={'variable': '','value': 'Number of Vehicles'}
                ).update_layout(showlegend=False).update_xaxes(title_text='Brand') ,
            ),
        ]),

        # Second plot
        html.Div(style={'width': '48%', 'background-color': '#ffffff', 'padding': '20px', 'border-radius': '10px', 'box-shadow': '0px 0px 10px #ccc', 'text-align': 'center'}, children=[
            html.H3("Average Price Over Year of Production", style={'color': '#a64647'}),
            dcc.Graph(
                id='average-price-plot',
                figure=px.line(
                    t,####
                    x='brand year',
                    y='price',
                    labels={'price': 'Average Price'},
                    color_discrete_sequence=['#8cb5d3']
                      ).update_xaxes(title_text='Year of Production'),
            ),
        ]),
    ]),

    # Model Performance
    html.Div(style={'background-color': '#ffffff', 'padding': '20px', 'border-radius': '10px', 'box-shadow': '0px 0px 10px #ccc', 'margin-top': '30px', 'text-align': 'center'}, children=[
        html.H2("Model Performance on Test Data", style={'color': '#1f3f66'}),
         html.Div([
        html.Table([
            html.Tr([html.Th("Brand"), html.Th("Year"), html.Th("Mileage"), html.Th("Accident History"), html.Th("Drive Mode"), html.Th("Actual Price"), html.Th("Predicted Price")]),
            *[
                html.Tr([
                    html.Td(f[0]),  # Brand
                    html.Td(f[1]),  # Mileage
                    html.Td(f[2]),  # Age
                    html.Td(f[3]),  # Accident History
                    html.Td(f[4]),  # Drive Mode
                    html.Td(t),
                    html.Td(p)
                ]) for f, t, p in zip(raw_features[:5].values, y_test[:5], y_pred_test[:5])
            ],
            ], style={'width': '100%', 'font-size': '16px'}),
            ], style={'margin-top': '20px'}),
        html.Div(style={'margin-top': '20px'}, children=[
            html.Table(style={'width': '20%', 'margin-left': 'auto', 'margin-right': 'auto'}, children=[
                html.Tr([html.Th("Mean Squared Error (log10)", style={'color': '#a64647'})]),
                html.Tr([html.Td(f"{8.634129}", style={'color': '#333'})]), #Mean Squared Error of Model on test set
            ])
        ]),
    ]),

    # Vehicle Price Estimator
    html.Div(style={'background-color': '#ffffff', 'padding': '20px', 'border-radius': '10px', 'box-shadow': '0px 0px 10px #ccc', 'margin-top': '30px', 'text-align': 'center'}, children=[
        html.H2("Vehicle Price Estimator", style={'color': '#1f3f66'}),
        html.Div(style={'width': '50%', 'margin-left': 'auto', 'margin-right': 'auto', 'text-align': 'left'}, children=[
          html.Label("Brand:", style={'font-size': '18px'}),
          dcc.Dropdown(
              id='brand_dropdown',
              options=[{'label': brand, 'value': brand} for brand in brands],
              # *******************************************************************
              value='Toyota',
              style={'width': '100%', 'font-size': '16px'}  #brand
          ),
          html.Label("Mileage:", style={'font-size': '18px'}),# mileage

          dcc.Input(id='mileage_input', type='number', value=50000, min=10000, max=120000, style={'width': '100%', 'font-size': '16px'}),
          html.Br(),
          html.Label("Year of production:", style={'font-size': '18px'}),#brand year
          #dcc.Input(id='age-input', type='number', value=2013, style={'width': '100%', 'font-size': '16px'}),
          dcc.Dropdown(
              id='age_input',
              options=[{'label': year, 'value': year} for year in [2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]],
              # *******************************************************************
              value=2015,
              style={'width': '100%', 'font-size': '16px'}
          ),
          html.Label("Accident History:", style={'font-size': '18px'}),# Accident history
          dcc.Dropdown(
              id='accident_dropdown',
              options=[
                  {'label': 'No Accident', 'value': False},
                  {'label': 'Accident', 'value': True}
                  # ****************************************************
              ],
              value=False,
              style={'width': '100%', 'font-size': '16px'}
          ),
          html.Label("Drive Mode:", style={'font-size': '18px'}),  # drive mode
          dcc.Dropdown(
              id='drive_mode_dropdown',
              options=[
                  {'label': '4WD', 'value': '4WD'},
                  {'label': 'AWD', 'value': 'AWD'},
                  {'label': 'FWD', 'value': 'FWD'},
                  {'label': 'RWD', 'value': 'RWD'}
                  # *************************************************************************
              ],
              value='4WD',
              style={'width': '100%', 'font-size': '16px'}
          ),
          html.Button('Predict Price', id='predict_button', style={'background-color': '#a64647', 'color': '#ffffff', 'font-size': '16px', 'margin-top': '10px'}),
          html.Div(id='result_output', style={'text-align': 'center', 'margin-top': '20px', 'color': '#333', 'font-size': '18px'}),
      ]),
    ]),
])

# callback function, process the input value and return the result
@app.callback(
    Output('result_output', 'children'),
    [Input('predict_button', 'n_clicks')],
    [dash.dependencies.State('brand_dropdown', 'value'),
     dash.dependencies.State('mileage_input', 'value'),
     dash.dependencies.State('age_input', 'value'),
     dash.dependencies.State('accident_dropdown', 'value'),
     dash.dependencies.State('drive_mode_dropdown', 'value')]
)

def predict_price(n_clicks, brand_dropdown, mileage_input, age_input, accident_dropdown, drive_mode_dropdown):
    # brand_input = le.transform([brand_dropdown])[0]
    brand_f = encoders['brand'].transform(pd.DataFrame({'brand':np.array(brand_dropdown)},index=[0])).toarray()
    if accident_dropdown:
      accident_f = encoders['accident history'].transform(pd.DataFrame({'accident history':np.array('At least 1 accident or damage reported')},index=[0])).toarray()
    else:
      accident_f = encoders['accident history'].transform(pd.DataFrame({'accident history':np.array('None reported')},index=[0])).toarray()
    drive_f = encoders['drive mode'].transform(pd.DataFrame({'drive mode':np.array(drive_mode_dropdown+' ')},index=[0])).toarray()
    fuelmin_f = parameters['fuel consumptiion min']
    fuelmax_f = parameters['fuel consumptiion max']
    fueltype_f = encoders['fuel type'].transform(pd.DataFrame({'fuel type':np.array(parameters['fuel type'])},index=[0])).toarray()
    one_f = encoders['one owner?'].transform(pd.DataFrame({'one owner?':np.array(parameters['one owner?'])},index=[0])).toarray()
    personal_f = encoders['personal use'].transform(pd.DataFrame({'personal use':np.array(parameters['personal use'])},index=[0])).toarray()
    newused_f = parameters['new/used']
    mileage = (mileage_input - mileage_mean)/mileage_std
    year = (age_input - year_mean)/year_std
    new_sample = np.concatenate(([[newused_f,mileage,year,fuelmin_f,fuelmax_f]],drive_f,fueltype_f,accident_f,one_f,personal_f,brand_f),axis=1)
    X_tensor = torch.tensor(new_sample,dtype=torch.float32).to(device)
    dataset = TensorDataset(X_tensor)
    dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

    model.eval()
    with torch.inference_mode():
      for batch in dataloader:
        X_test = batch
        predicted_price = model(X_test[0])

    result_output = f'Predicted Price: ${predicted_price.tolist()[0][0]:,.2f}'

    # return the predicted result
    return html.Div([
        html.H3(result_output, style={'color': '#a64647'})
    ])

if __name__ == '__main__':
    #app.run_server(debug=True)
    app.run_server(debug=True, host='127.0.0.1', port=8050)

c:\Users\muyan\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

c:\Users\muyan\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

c:\Users\muyan\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

c:\Users\muyan\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

c:\Users\muyan\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `